In [4]:
import csv
from biomart import BiomartServer
#import time
import pandas as pd

In [5]:
# DICTIONARY
dictionary = {}

def gene_dictionary(index,
                    gene_name,
                    cluster,
                    gene_list_id,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    alias,
                    direction):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Cluster_id' : cluster,
        'Gene_list_id' : gene_list_id,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction
    }
    
    return dictionary



# REGULATION
def regulation(gene_name):
    if gene_name in downTab:
        return 'Down'
    elif gene_name in upTab:
        return 'Up'
    else:
        return 'Unknown'
    

# RESPONSE  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'mgi_symbol':[mgi_symbol]}                     # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

# response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
# if values not return ????

    return values  

def biomartHumanOrthologs(mgi_symbol):
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'mgi_symbol':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
# response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
# if values not return ????

    return values


In [6]:
# clusters_file
f = open('./Clusters_-_Symbols.tsv')
tsv_file = csv.reader(f, delimiter="\t")

# biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']


# separation_of_headers  
headers = next(tsv_file)               
rest2 = list(tsv_file)

# separation_of_regulation
downTab = [row[16] for row in rest2]
upTab = [row[17] for row in rest2]

# separation_of_rest
[row.pop(17) for row in rest2]   
[row.pop(16) for row in rest2]

rest = []
for row in rest2:
    for i in range(len(row)):
        if row[i] not in rest:
            rest.append(row[i])
            
#print(rest2)
#print(rest)

In [7]:
# CREATING FINAL LIST
ls_geneDictionaries = []
ls_biomartParameters = []
ls_biomartHumanOrthologs = []
ls_notResponse = []


for i in range(len(rest)):
    gene_name = rest[i]
#   print(gene_name)
    if gene_name != '':

        # variables

        geneSource = 'marpiech_tissues_dex'
        cluster = headers[i % 16]
        gene_list_id = geneSource + '_' + cluster
        index = i + 1
        direction = regulation(gene_name)          # => info
        alias = ''


        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters = biomartParameters(gene_name)
        if not ls_biomartParameters:
            ls_notResponse.append(gene_name)
#            print(gene_name)


        for ls in ls_biomartParameters:
            for j in range(3):
                if ls[j] == '':
                    ls[j] = ''
                    
            ensembl_gene_id_temp.append(ls[0]) 
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])

        ensembl_gene_id = '|'.join(list(set(ensembl_gene_id_temp)))
        ensembl_transcript_id = '|'.join(list(set(ensembl_transcript_id_temp)))
        refseq_mrna = '|'.join(list(set(refseq_mrna_temp)))


        ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        


        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               cluster,
                                               gene_list_id,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               alias,
                                               direction)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)

data = ls_geneDictionaries

df = pd.DataFrame.from_dict(data)



"""
for gene in ls_geneDictionaries:
print(gene)
"""



{'Index': 1, 'Gene_name': 'Mef2c', 'Cluster_id': '1', 'Gene_list_id': 'marpiech_tissues_dex_1', 'Ensembl_gene_id': 'ENSMUSG00000005583', 'Ensembl_transcript_id': 'ENSMUST00000198064|ENSMUST00000198360|ENSMUST00000163888|ENSMUST00000199262|ENSMUST00000199105|ENSMUST00000198199|ENSMUST00000199167|ENSMUST00000198217|ENSMUST00000197938|ENSMUST00000197145|ENSMUST00000199019|ENSMUST00000196832|ENSMUST00000198069|ENSMUST00000197022|ENSMUST00000199210|ENSMUST00000196730|ENSMUST00000005722|ENSMUST00000197722|ENSMUST00000197681|ENSMUST00000196207|ENSMUST00000185052|ENSMUST00000200394|ENSMUST00000195904|ENSMUST00000200138|ENSMUST00000198916|ENSMUST00000200123|ENSMUST00000197146|ENSMUST00000195984|ENSMUST00000199432|ENSMUST00000199450|ENSMUST00000196493', 'RefSeq_mRNA_id': '|NM_001347576|NM_001347579|NM_001170537|NM_001347566|NM_001347574|NM_001347577|NM_001347568|NM_001347578|NM_001347572|NM_025282|NM_001347575|NM_001347580|NM_001347564|NM_001347569|NM_001347571|NM_001347581|NM_001347573|NM_00134

'\nfor gene in ls_geneDictionaries:\nprint(gene)\n'

In [8]:
print(display(df))

,Index,Gene_name,Cluster_id,Gene_list_id,Ensembl_gene_id,Ensembl_transcript_id,RefSeq_mRNA_id,HGNC_symbol,Alias,Info
0,1,Mef2c,1,marpiech_tissues_dex_1,ENSMUSG00000005583,ENSMUST00000198064|ENSMUST00000198360|ENSMUST0...,|NM_001347576|NM_001347579|NM_001170537|NM_001...,MEF2C,,Down
1,2,Cxcl1,2,marpiech_tissues_dex_2,ENSMUSG00000029380,ENSMUST00000031327|ENSMUST00000201245,|NM_008176,,,Down
2,3,Id1,3,marpiech_tissues_dex_3,ENSMUSG00000042745,ENSMUST00000109824|ENSMUST00000038368,NM_010495|NM_001355113|NM_001369018,ID1,,Down
3,4,Aplnr,4,marpiech_tissues_dex_4,ENSMUSG00000044338,ENSMUST00000184728|ENSMUST00000057019,|NM_011784,APLNR,,Down
4,5,Phlda2,5,marpiech_tissues_dex_5,ENSMUSG00000010760,ENSMUST00000010904|ENSMUST00000207425|ENSMUST0...,|NM_009434,PHLDA2,,Down
...,...,...,...,...,...,...,...,...,...,...
474,476,Polr3e,12,marpiech_tissues_dex_12,ENSMUSG00000030880,ENSMUST00000207481|ENSMUST00000033173|ENSMUST0...,|NM_025298|NM_001379288|NM_001164096,POLR3E,,Up
475,477,Arhgef28,13,marpiech_tissues_dex_13,ENSMUSG00000021662,ENSMUST00000238746|ENSMUST00000225884|ENSMUST0...,NM_012026||NM_001408684|NM_001408685|NM_001408686,ARHGEF28,,Up
476,478,Fgfbp1,14,marpiech_tissues_dex_14,ENSMUSG00000048373,ENSMUST00000199894|ENSMUST00000199481|ENSMUST0...,|NM_001271616|NM_008009,FGFBP1,,Up
477,479,Uap1,15,marpiech_tissues_dex_15,ENSMUSG00000026670,ENSMUST00000160848|ENSMUST00000027981|ENSMUST0...,|NM_001305045|NM_001305046|NM_133806,UAP1,,Up


None


In [9]:
df.to_excel(r'C:\Users\Adrian\Desktop\ASIA\withHGNC.xlsx', index=False)

In [10]:
df.to_csv('example.tsv', sep="\t")

In [15]:
count = 0
for ls in ls_notResponse:
    print(count, ls)
    count += 1

0 6620401C13Rik
1 9330132O05Rik
2 9530053J19Rik
3 Gpr81
4 A830037N18Rik
5 9430023L20Rik
6 Cyr61
7 Gm6484
8 CKLF5
9 9830132E05Rik
10 Niacr1
11 LOC332300
12 LOC381739
13 LOC100040353
14 LOC672274
15 6030408B16Rik
16 LOC628050
17 3110023G01Rik
18 Nrp
19 Ccrn4l
20 1200012P04Rik
21 A530020H22Rik
22 9930013L23Rik
23 Ctgf
24 EG245297
25 D030063E12
26 9330177P18Rik
27 LOC624610
28 Fam129b
29 1700081H05Rik
30 4930544G21Rik
31 LOC100047902
32 Mllt4
33 Lrrc16
34 Grasp
35 LOC665946
36 Hmga2-ps1
37 BC020108
38 4632428N05Rik
39 Egfbp2
40 4930502E18Rik
41 D14Ertd668e
42 A530026G17
43 Mela
44 BC030863
45 LOC100041932
46 A130038J17Rik
47 1810011O10Rik
48 8430408G22Rik
49 Amica1
50 EG668311
51 EG640530
